In [13]:
import numpy as np
import pandas as pd
import ast
import collections

In [80]:
exp_num = 6
df = pd.read_csv('sankey-helper-{}.csv'.format(exp_num), \
                 converters={'byhost':ast.literal_eval, 'byvname': ast.literal_eval})

In [81]:
df[:5]

,namevalue,context_id,time_stamp,byhost,byvname
0,"('131', '20200502')",1,6,{match.deepintent.com},"{forbes, theguardian}"
1,"('131', '20200502')",2,1,{.deepintent.com},{medscape}
2,"('A3', 'd=AQABBKPMrF4CEKCi8LpQuotbEk2A-M-xq1YF...",1,21,"{cms.analytics.yahoo, pr.ybp.yahoo, pr-bh.ybp....","{cnn, nytimes, forbes, theguardian}"
3,"('A3', 'd=AQABBKPMrF4CEKCi8LpQuotbEk2A-M-xq1YF...",2,44,"{ads.yahoo.com, cookiex.ngd.yahoo, service.ids...","{verywellhealth, webmd, medscape}"
4,"('A3', 'd=AQABBKPMrF4CEKCi8LpQuotbEk2A-M-xq1YF...",3,12,"{sp.analytics.yahoo, .yahoo.com}",{colledgeboard}


In [82]:
keywords = """deepintent
yahoo
advertising
ib-ibi
mookie1
atdmt
casalemedia
exelator
bttrack
doubleclick
contextweb
nr-data
pubmatic
bing
adsrvr
tapad
adsymptotic
scorecardresearch
crwdcntrl
districtm
krxd
1rx
agkn
amazon-adsystem
adentifi
adnxs
spotxchange
rubiconproject
bidr
bluekai
dnacdn
bidswitch
myvisualiq
ipredictive
quantserve
media.net
demdex
everesttech
extend.tv
facebook
openx
mathtag
mxptint
twitter
rlcdn
rkdms
owneriq
sitescout
sharethrough
simpli
taboola
3lift
myvisualiq
turn.com
adform
w55c
zemanta
tribalfusion
linkedin
iasds01
dotomi
appier
acuityplatform
advangelists
mfadsrvr
zorosrv
smartadserver
rfihub
srv
creativecdn
addthis
gumgum
yieldmo"""
keywords = keywords.split("\n")

In [83]:
nv_ls = df.namevalue.tolist()
origin_hosts = df.byhost.tolist()
origin_web = df.byvname.tolist()
contexts_ls = df.context_id.tolist()

In [84]:
ls_news = ['cnn','nytimes','theguardian','washingtonpost','forbes']
ls_health = ['webmd','mayoclinic','myfitnesspal','medscape','verywellhealth']
ls_education = ['jhu','mit','edx','harvard','colledgeboard']
name_dict = {1: ls_news, 2: ls_health, 3:ls_education}
order_ls = [[1,2,3],[2,1,3],[3,1,2],[3,2,1],[1,3,2],[2,3,1]]
order = order_ls[exp_num-1]
l = name_dict[order[0]] + name_dict[order[1]] + name_dict[order[2]]

In [85]:
dict_ls = collections.defaultdict(set)

In [86]:
for i in range(1, len(nv_ls)):
    if nv_ls[i-1] != nv_ls[i]:
        continue
    u_hosts = origin_hosts[i].union(origin_hosts[i-1])
    if order.index(contexts_ls[i]) < order.index(contexts_ls[i-1]):
        in_webs = origin_web[i]
        out_webs = origin_web[i-1]
    else:
        in_webs = origin_web[i-1]
        out_webs = origin_web[i]
    for domain in keywords:
        if any(domain in string for string in u_hosts):
            for in_web in in_webs:
                for out_web in out_webs:
                    dict_ls[in_web].add((out_web, domain))

In [87]:
inbounds = []
outbounds = []
trackers = []

In [88]:
for key, s in dict_ls.items():
    for outbound, tracker in s:
        inbounds.append(key)
        outbounds.append(outbound)
        trackers.append(tracker)


In [89]:
df_2 = pd.DataFrame({"exp":[exp_num]*len(inbounds), "from":inbounds, "to":outbounds, "by": trackers})

In [90]:
df_2.shape

(500, 4)

In [91]:
big_df = pd.concat([big_df, df_2])

In [92]:
big_df.shape

(1674, 4)

In [94]:
df_2[:10]

,exp,from,to,by
0,6,medscape,washingtonpost,advertising
1,6,medscape,cnn,yahoo
2,6,medscape,theguardian,acuityplatform
3,6,medscape,forbes,sharethrough
4,6,medscape,washingtonpost,openx
5,6,medscape,washingtonpost,yahoo
6,6,medscape,nytimes,adnxs
7,6,medscape,theguardian,deepintent
8,6,medscape,nytimes,rfihub
9,6,medscape,theguardian,contextweb


In [95]:
big_df.to_csv("linkage_details.csv", index=False)

In [104]:
big_df_sum = big_df.groupby(['exp', 'from', 'to'])['by'].apply(set).reset_index(name='by')

In [105]:
big_df_sum.to_csv("linkage_details_with_host_ls.csv", index=False)